In [8]:
from find_source import summary, significant
from astropy.io import fits
from astropy.coordinates import Angle, SkyCoord
import astropy.units as u
import glob

In [9]:
def make_catalog(fits_file: str):
    '''
    Summarizes information on any significant point sources detected in an image.

    Parameters
    ----------
    fits_file : str
        The path of the FITS file that contains the image.

    Returns
    -------
    dict
        A dictionary with:
            dict(s)
                A dictionary with:
                    str
                        The name of the target object of the observation.
                    str
                        The date and time of the observation.
                    str
                        The name of the FITS file with the image.
                    Angle
                        The restoring beam major axis.
                    Angle
                        The restoring beam minor axis.
                    Angle
                        The restoring beam position angle.
                    float
                        The uncertainty in flux density measurements. The rms excluding any significant sources and a small circular region around them.
                    float
                        The flux density of the detected point source.
                    SkyCoord
                        The location of the detected point source.
                    bool
                        Whether the detected point source is in the initial search region.
    '''

    summ = summary(fits_file, True, False, False)

    header_data = fits.getheader(fits_file)
    name = header_data['OBJECT']
    obs_date_time = header_data['DATE-OBS']
    bmaj = header_data['BMAJ']
    bmin = header_data['BMIN']
    bpa = header_data['BPA']
    ctype1 = header_data['CTYPE1']
    crval1 = header_data['CRVAL1']
    cunit1 = header_data['CUNIT1']
    ctype2 = header_data['CTYPE2']
    crval2 = header_data['CRVAL2']
    cunit2 = header_data['CUNIT2']

    #assume beam axes in same units as CUNIT1 and CUNIT2 and BPA in degrees
    beam_maj_axis = Angle(bmaj, cunit1)
    beam_min_axis = Angle(bmin, cunit1)
    beam_pos_angle = Angle(bpa, u.degree)

    interesting_sources = {}
    field_info = {'field_name': name, 'obs_date_time': obs_date_time, 'file_name': fits_file[fits_file.rindex('/')+1:],\
                           'beam_maj_axis': beam_maj_axis, 'beam_min_axis': beam_min_axis, 'beam_pos_angle': beam_pos_angle,\
                           'flux_uncertainty': summ['rms']}

    n_ext_sources = 0
    if type(summ['ext_peak_val']) == list:
        n_ext_sources += len(summ['ext_peak_val'])

    ra_index = 0
    dec_index = 1

    if 'RA' in ctype1:
        ra = crval1
    elif 'RA' in ctype2:
        ra = crval2
        ra_index = 1
    else:
        raise ValueError('No RA in image')

    if 'DEC' in ctype1:
        dec = crval1
        dec_index = 0
    elif 'DEC' in ctype2:
        dec = crval2
    else:
        raise ValueError('No dec in image')

    if cunit1 != cunit2:
        raise ValueError('Axes have different units')

    center = SkyCoord(ra, dec, unit=cunit1)

    pt_source_count = 1

    if significant(fits_file):
        int_info = field_info.copy()
        int_info['flux_density'] = summ['int_peak_val']

        int_ra_offset = summ['int_peak_coord'][ra_index] * u.arcsec
        int_dec_offset = summ['int_peak_coord'][dec_index] * u.arcsec
        int_info['coord'] = center.spherical_offsets_by(int_ra_offset, int_dec_offset)

        int_info['internal'] = True

        key = f'source_{pt_source_count}'
        interesting_sources[key] = int_info
        pt_source_count +=1

    for i in range(n_ext_sources):
        ext_info = field_info.copy()
        ext_info['flux_density'] = summ['ext_peak_val'][i]

        ext_ra_offset = summ['ext_peak_coord'][i][ra_index] * u.arcsec
        ext_dec_offset = summ['ext_peak_coord'][i][dec_index] * u.arcsec
        ext_info['coord'] = center.spherical_offsets_by(ext_ra_offset, ext_dec_offset)

        ext_info['internal'] = False

        key = f'source_{pt_source_count}'
        interesting_sources[key] = ext_info
        pt_source_count += 1

    if 'source_1' not in interesting_sources:
        return
    else:
        return interesting_sources

In [ ]:
def combine_catalogs(catalog_1: dict, catalog_2: dict):
    '''
    Combines two catalogs in the format returned by make_catalog() into a single catalog of the same format.

    Parameters
    ----------
    catalog_1 : dict
        The catalog to which the other catalog will be "appended."
    catalog_2 : dict
        The catalog to "append" to the other catalog.

    Returns
    -------
    dict
        A dictionary of the combined catalogs in the same catalog format.
    '''

    shift = len(catalog_1)
    for key, value in catalog_2.items():
        new_number = int(key.replace('source_', ''))
        new_key = f'source_{new_number + shift}'
        catalog_1[new_key] = value
    return catalog_1

In [11]:
final_catalog = {}
for file in glob.glob('../data/multi_track/*.fits'):
    try:
        info = make_catalog(file)
        if info != None:
            combine_catalogs(final_catalog, info)
    except:
        print(f'Try again for {file}')

print(final_catalog)

/Users/chloexu/conda/anaconda3/envs/casaenv/lib/python3.10/site-packages/astropy/units/quantity.py:1355: ComplexWarning: Casting complex values to real discards the imaginary part
  return float(self.to_value(dimensionless_unscaled))


Try again for ../data/multi_track/0854+281_11145.fits
{'source_1': {'field_name': '3c84', 'obs_date_time': '4-7-25 10:51:35', 'file_name': '3c84_11028.fits', 'beam_maj_axis': <Angle 0.00114532 deg>, 'beam_min_axis': <Angle 0.00088713 deg>, 'beam_pos_angle': <Angle 108.0581 deg>, 'flux_uncertainty': 0.015656273812055588, 'flux_density': 8.948379516601562, 'coord': <SkyCoord (ICRS): (ra, dec) in deg
    (49.95044899, 41.51153311)>, 'internal': True}, 'source_2': {'field_name': '3c84', 'obs_date_time': '4-7-25 10:51:35', 'file_name': '3c84_11028.fits', 'beam_maj_axis': <Angle 0.00114532 deg>, 'beam_min_axis': <Angle 0.00088713 deg>, 'beam_pos_angle': <Angle 108.0581 deg>, 'flux_uncertainty': 0.015656273812055588, 'flux_density': 0.12322600930929184, 'coord': <SkyCoord (ICRS): (ra, dec) in deg
    (49.95371361, 41.50175322)>, 'internal': False}, 'source_3': {'field_name': '3c84', 'obs_date_time': '4-7-25 10:51:35', 'file_name': '3c84_11028.fits', 'beam_maj_axis': <Angle 0.00114532 deg>, 'b